In [2]:
#Add needed imports
import numpy as np
import pandas as pd
import os
import shap
from numpy import mean
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import RobustScaler
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier  
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow import keras
import numpy as np
import keras 
import pydot
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from keras.utils import plot_model
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
import os
import shap

from tensorflow.compat.v1.keras.backend import get_session
tf.compat.v1.disable_v2_behavior()
#Read data
proccessed_data_path =os.path.join(os.path.pardir,os.path.pardir,'data','processed')
train_path = os.path.join(proccessed_data_path,'dataset7.csv')
df = pd.read_csv(train_path)
labels=df['Churn']
x = df.drop(columns=['Churn','Unnamed: 0'],axis = 'columns')
y=np.ravel(labels)
oversample = SMOTENC(categorical_features=[9,10,11,12])
x, y = oversample.fit_resample(x, y)
sc = RobustScaler()
x = pd.DataFrame(sc.fit_transform(x),columns = x.columns)
ii = 1

xgb_model = XGBClassifier(random_state=1,learning_rate=0.05, max_depth=7,eval_metric='mlogloss',use_label_encoder =False ,objective="binary:logistic")
dt_model=DecisionTreeClassifier(random_state=1,criterion='entropy',max_depth = 7,min_samples_leaf=30)  
nn_model = Sequential()
nn_model.add(Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.001), input_dim=13, activation='relu' ))
nn_model.add(Dropout(rate=0.2))
nn_model.add(Dense(8,kernel_regularizer=tf.keras.regularizers.l2(0.001),activation='relu'))
nn_model.add(Dropout(rate=0.1))
nn_model.add(Dense(1, activation='sigmoid'))

callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc',patience=70,restore_best_weights=True)
cv = StratifiedKFold(n_splits=10)
for train_index, test_index in cv.split(x, y):
	print("Iteration" , ii)
	Xi_train, Xi_test = x.loc[train_index], x.loc[train_index]
	yi_train, yi_test = y[train_index], y[test_index]
	xgb_model.fit(Xi_train,yi_train)
	shap_values = shap.TreeExplainer(xgb_model).shap_values(Xi_test,approximate=True)
	#shap.summary_plot(shap_values, Xi_test, plot_type="bar")
	vals = np.abs(shap_values).mean(axis=0)
	print(pd.DataFrame(list(zip(vals)),columns=['XGB - feature_importance_vals']).to_string(index=False))    
	Xi_train, Xi_test = x.loc[train_index], x.loc[test_index]
	yi_train, yi_test = y[train_index], y[test_index]
	dt_model.fit(Xi_train,yi_train)
	explainer = shap.TreeExplainer(dt_model)
	shap_values = explainer.shap_values(Xi_test)
	#shap.summary_plot(shap_values[1], Xi_test, plot_type="bar")
	vals = np.abs(shap_values[1]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['DT - feature_importance_vals']).to_string(index=False))
	lr_schedule= tf.keras.optimizers.schedules.InverseTimeDecay( 0.001,decay_steps=(Xi_train.shape[0]/32)*50,decay_rate=1,staircase=False)
	nn_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr_schedule), metrics=['accuracy'])
	nn_model.fit(Xi_train, yi_train, validation_data=(Xi_test, yi_test), epochs=150, batch_size=10,verbose=0,callbacks=[callback])
	explainer = shap.DeepExplainer(nn_model, Xi_train[1:100].to_numpy())
	shap_values = explainer.shap_values(Xi_test[1:20].to_numpy())
	vals = np.abs(shap_values[0]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['NN - feature_importance_vals']).to_string(index=False))
	ii += 1                             

Iteration 1


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.277181
                      0.918175
                      0.638090
                      0.153218
                      0.224365
                      0.236383
                      0.336723
                      0.320482
                      0.145299
                      0.017546
                      0.011000
                      0.000000
                      0.000000
 DT - feature_importance_vals
                     0.020737
                     0.194729
                     0.120342
                     0.008754
                     0.016981
                     0.017578
                     0.028233
                     0.060580
                     0.014413
                     0.000000
                     0.000000
                     0.000000
                     0.000000
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


keras is no longer supported, please use tf.keras instead.


 NN - feature_importance_vals
                     0.016114
                     0.100653
                     0.028682
                     0.004591
                     0.050185
                     0.082485
                     0.044162
                     0.066672
                     0.015271
                     0.008022
                     0.009151
                     0.014386
                     0.005937
Iteration 2


...

 XGB - feature_importance_vals
                      0.343955
                      0.910255
                      0.671732
                      0.144258
                      0.209091
                      0.205903
                      0.362080
                      0.302249
                      0.150094
                      0.028560
                      0.004641
                      0.000000
                      0.000000
 DT - feature_importance_vals
                     0.035613
                     0.161956
                     0.119697
                     0.000000
                     0.019709
                     0.023817
                     0.050083
                     0.031508
                     0.042901
                     0.000000
                     0.000000
                     0.004620
                     0.000000
 NN - feature_importance_vals
                     0.047619
                     0.279620
                     0.122813
                     0.011

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.339278
                      0.911915
                      0.669930
                      0.130328
                      0.195543
                      0.238742
                      0.324285
                      0.361870
                      0.177919
                      0.033622
                      0.006728
                      0.000000
                      0.002855
 DT - feature_importance_vals
                     0.045477
                     0.179832
                     0.120742
                     0.012279
                     0.008795
                     0.032110
                     0.045456
                     0.064983
                     0.011204
                     0.000000
                     0.000000
                     0.000000
                     0.000000
 NN - feature_importance_vals
                     0.029002
                     0.243170
                     0.115910
                     0.021

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.355790
                      0.895777
                      0.667558
                      0.126430
                      0.218133
                      0.202238
                      0.333298
                      0.355166
                      0.177405
                      0.028353
                      0.002199
                      0.000000
                      0.002663
 DT - feature_importance_vals
                     0.031400
                     0.168478
                     0.116715
                     0.010732
                     0.006551
                     0.011571
                     0.053584
                     0.072521
                     0.017485
                     0.000000
                     0.000000
                     0.000000
                     0.005299
 NN - feature_importance_vals
                     0.013512
                     0.308121
                     0.093781
                     0.006